# Planning Pattern

<!-- <img src="../img/planning_pattern.png" alt="Alt text" width="500"/> -->

---

Planning basically means deciding what sequence of steps to follow to accomplish a large task.

That is exactly what the Planning Pattern provides; ways for the LLM to break a task into **smaller, more easily accomplished subgoals** without losing track of the end goal.

The most paradigmatic example of the planning pattern is the [**ReAct**](https://react-lm.github.io/) technique.

## Relevant imports and OpenAI Client

In [6]:
import os
import re
import sys
import math
import json
from dotenv import load_dotenv

from openai import OpenAI

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.tool_agent.tool import tool
from src.utils.extraction import extract_tag_content


load_dotenv()

model = "gpt-4o"
client = OpenAI()

## A System Prompt for the ReAct Loop

The difference in the System Prompt is that it describes the ReAct loop, so that the LLM is aware of
the three operations it's allowed to use:

1. Thought: The LLM will think about which action to take
2. Action: The LLM will use a Tool to "act on the environment"
3. Observation: The LLM will observe the tool output and reflect on the next thing to do.

We enclose all the messages with tags, like  <thought></thought>, <observation></observation>. This way it is easier for the LLM to understand the instructions this way.


In [7]:
# Define the System Prompt as a constant
REACT_SYSTEM_PROMPT = """
You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
%s
</tools>

Example session:

<question>What's the current temperature in Madrid?</question>
<thought>I need to get the current weather in Madrid</thought>
<tool_call>{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}, "id": 0}</tool_call>

You will be called again with this:

<observation>{0: {"temperature": 25, "unit": "celsius"}}</observation>

You then output:

<response>The current temperature in Madrid is 25 degrees Celsius</response>

Additional constraints:

- If the user asks you something unrelated to any of the tools above, answer freely enclosing your answer with <response></response> tags.
"""

## Example step by step

### Defining the Tools

Let's build an example that involves the use of three tools, like the following ones.

In [8]:
@tool
def sum_two_elements(a: int, b: int) -> int:
    """
    Computes the sum of two integers.

    Args:
        a (int): The first integer to be summed.
        b (int): The second integer to be summed.

    Returns:
        int: The sum of `a` and `b`.
    """
    return a + b


@tool
def multiply_two_elements(a: int, b: int) -> int:
    """
    Multiplies two integers.

    Args:
        a (int): The first integer to multiply.
        b (int): The second integer to multiply.

    Returns:
        int: The product of `a` and `b`.
    """
    return a * b

@tool
def compute_log(x: int) -> float | str:
    """
    Computes the logarithm of an integer `x` with an optional base.

    Args:
        x (int): The integer value for which the logarithm is computed. Must be greater than 0.

    Returns:
        float: The logarithm of `x` to the specified `base`.
    """
    if x <= 0:
        return "Logarithm is undefined for values less than or equal to 0."
    
    return math.log(x)


available_tools = {
    "sum_two_elements": sum_two_elements,
    "multiply_two_elements": multiply_two_elements,
    "compute_log": compute_log
}

In [9]:
print("Tool name: ", sum_two_elements.name)
print("Tool signature: ", sum_two_elements.fn_signature)

Tool name:  sum_two_elements
Tool signature:  {"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}


### Adding the Tools signature to the System Prompt

Now, we just concatenate the tools signature and add them to the System Prompt.

In [10]:
tools_signature = sum_two_elements.fn_signature + ",\n" + multiply_two_elements.fn_signature + ",\n" + compute_log.fn_signature

In [11]:
print(tools_signature)

{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "multiply_two_elements", "description": "\n    Multiplies two integers.\n\n    Args:\n        a (int): The first integer to multiply.\n        b (int): The second integer to multiply.\n\n    Returns:\n        int: The product of `a` and `b`.\n    ", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}},
{"name": "compute_log", "description": "\n    Computes the logarithm of an integer `x` with an optional base.\n\n    Args:\n        x (int): The integer value for which the logarithm is computed. Must be greater than 0.\n\n    Returns:\n        float: The logarithm of `x` to the specified `base`.\n    ", "parameters": {"prop

In [12]:
REACT_SYSTEM_PROMPT = REACT_SYSTEM_PROMPT % tools_signature

In [9]:
print(REACT_SYSTEM_PROMPT)


You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
{"name": "sum_two_elements", "description": "\n    Computes the sum of two integers.\n\n    Args:\n        a (int): The first integer to be summed.\n        b (int): The second integer to be summed.\n\n    Returns:\n        int: The sum of `a` and `b`.\n    ", "parameters": {"pr

### ReAct Loop Step 1

In [13]:
USER_QUESTION = "I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result"
chat_history = [
    {
        "role": "system",
        "content": REACT_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": f"<question>{USER_QUESTION}</question>"
    }
]


In [12]:
output = client.chat.completions.create(
    messages=chat_history,
    model=model
).choices[0].message.content

print(output)

APITimeoutError: Request timed out.

In [14]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

NameError: name 'output' is not defined

### ReAct Loop Step 2

In [15]:
tool_call = extract_tag_content(output, tag="tool_call")

NameError: name 'output' is not defined

In [16]:
tool_call

NameError: name 'tool_call' is not defined

In [15]:
tool_call = json.loads(tool_call.content[0])

NameError: name 'tool_call' is not defined

In [ ]:
tool_call

In [16]:
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

NameError: name 'tool_call' is not defined

In [ ]:
assert tool_result == 1234 + 5678

In [17]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

NameError: name 'tool_result' is not defined

### ReAct Loop Step 3

In [18]:
output = client.chat.completions.create(
    messages=chat_history,
    model=model
).choices[0].message.content

print(output)

APITimeoutError: Request timed out.

In [ ]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

### ReAct Loop Step 4

In [ ]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [ ]:
tool_result

In [ ]:
assert tool_result == (1234 + 5678) * 5

In [ ]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

### ReAct Loop Step 5

In [ ]:
output = client.chat.completions.create(
    messages=chat_history,
    model=model
).choices[0].message.content

print(output)

In [ ]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

### ReAct Loop Step 6

In [ ]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

In [ ]:
tool_result

In [ ]:
assert tool_result == math.log((1234 + 5678) * 5)

In [ ]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

### ReAct Loop Step 7

In [ ]:
output = client.chat.completions.create(
    messages=chat_history,
    model=model
).choices[0].message.content

print(output)

## Doing the same but with `agentic_patterns` library

In [19]:
from src.planning_agent.react_agent import ReactAgent

In [20]:
agent = ReactAgent(tools=[sum_two_elements, multiply_two_elements, compute_log])

In [22]:
agent.run(user_msg="I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result")

APITimeoutError: Request timed out.